In [1]:
import os

from tqdm import tqdm

import numpy as np

from bs4 import BeautifulSoup

from scipy.sparse import dok_matrix

from collections import defaultdict

## PageRank

In [2]:
directory = './raw'

files = os.listdir(directory)
stored_pages = [page.replace('.html', '') for page in files]

all_pages = list()

for file in tqdm(files):
    filename = os.path.join(directory, file)

    with open(filename, 'r') as f:
        content = f.read()
        soup = BeautifulSoup(content, 'lxml')

    titles_on_page = list()
    for a_tag in soup.find_all('a'):
        title = a_tag.get('title', '')
        title = title.replace('_', ' ')
        if title in stored_pages:
            if title not in titles_on_page:
                titles_on_page.append(title)

    all_pages.append(titles_on_page)

100%|██████████| 2379/2379 [01:52<00:00, 21.19it/s]


In [5]:
size = len(all_pages)
rank_matrix = dok_matrix((size, size), dtype='float32') # Матрица переходов

for main_index, main_page in tqdm(enumerate(all_pages), total=size):
    for index, title in enumerate(main_page):
        if title in stored_pages:
            rank_matrix[index, main_index] = 1

100%|██████████| 2379/2379 [00:00<00:00, 3366.32it/s]


In [8]:
norm_matrix = dok_matrix((size, size), dtype='float32') # Матрица близости

for index, row in enumerate(rank_matrix):
    norm_matrix[index] = row / row.sum()

In [11]:
random_walk = np.ones(size, dtype='float32') / size
random_walk

array([0.00042034, 0.00042034, 0.00042034, ..., 0.00042034, 0.00042034,
       0.00042034], dtype=float32)

In [12]:
# Считаем PageRank

def calculate_page_vector(norm_matrix, a=0.80, eps = 1e-20, max_iteration=1000):
    new_vector_state = np.ones(size, dtype='float32') / size
    norm_M_hat = (a / norm_matrix.shape[0]) + (((1 - a) / len(norm_matrix) * norm_matrix))

    for index in range(max_iteration):
        last_vector_state = new_vector_state
        new_vector_state = np.matmul(last_vector_state, norm_M_hat)
        
        diff = np.abs((last_vector_state - new_vector_state)).sum()
        if diff <= eps:
            print(f'Последняя разница {diff} на шаге {index}')
            return new_vector_state

    return None

pagerank_vector = calculate_page_vector(norm_matrix.toarray())

Последняя разница 8.306609103568592e-21 на шаге 200


In [13]:
## Проверяем количество ненулевых элементов в векторе
len([elem for elem in pagerank_vector if elem > 0])

2379

In [14]:
PRINT_TOP = 10

most_paged_elements = np.argsort(pagerank_vector)[::-1]

most_paged = list()
most_paged_armstrong = list()

for elem_index in most_paged_elements:
    elem = stored_pages[elem_index]
    if 'Армстронг' in elem and len(most_paged_armstrong) <= PRINT_TOP:
        most_paged_armstrong.append(elem)

    if len(most_paged) <= PRINT_TOP:
        most_paged.append(elem)

    if len(most_paged) >= PRINT_TOP and len(most_paged_armstrong) >= PRINT_TOP:
        break

In [15]:
print('Топ обычных страниц:')
for page in most_paged:
    print(page)

print()
print('Топ страниц про Армстронгов:')
for page in most_paged_armstrong:
    print(page)

Топ обычных страниц:
Английский_язык
США
Соединённые_Штаты_Америки
1_июля
22_октября
6_февраля
1_января
22_июня
2_ноября
Военный_министр_США
4_марта

Топ страниц про Армстронгов:
Армстронг,_Лэнс
Армстронг,_Луи
Билли_Джо_Армстронг
Армстронг,_Билли_Джо
Армстронг,_Нил_Олден
Армстронг,_Нил
Армстронг,_Джерри
Армстронг,_Самира
Армстронг_(округ,_Пенсильвания)
Армстронг,_Тим


## HITS

In [16]:
def hits(A):
    AAt = np.dot(A, A.T)
    AtA = np.dot(A.T, A)

    h_eig_value, h = np.linalg.eig(AAt)
    h_eig_value_argmax = np.argmax(h_eig_value)
    h = np.real(h[:,h_eig_value_argmax]).astype(float)

    a_eig_value, a = np.linalg.eig(AtA)
    a_eig_value_argmax = np.argmax(a_eig_value)
    a = np.real(a[:,a_eig_value_argmax]).astype(float)

    return a, h

In [17]:
M = norm_matrix.toarray()

armstrong_indexes = [stored_pages.index(k) for k in stored_pages if "Армстронг" in k]

A = np.zeros(M.shape)
A[armstrong_indexes] = M[armstrong_indexes]
A[:,armstrong_indexes] = M[:,armstrong_indexes]

authority, hubs = hits(A)
authority = authority[armstrong_indexes]
hubs = hubs[armstrong_indexes]

sort_buf = np.array([authority, armstrong_indexes]).T
sort_buf = sort_buf[sort_buf[:,0].argsort()]
idx_authority_sort = sort_buf[:,1][::-1].astype(int)

print("Топ Authority: Армстронг")
for i in range(10):
    print(stored_pages[idx_authority_sort[i]])

print()

sort_buf = np.array([hubs, armstrong_indexes]).T
sort_buf = sort_buf[sort_buf[:,0].argsort()]
idx_hub_sort = sort_buf[:,1][::-1].astype(int)

print("Топ Hub: Армстронг")
for i in range(10):
    print(stored_pages[idx_hub_sort[i]])

Топ Authority: Армстронг
Армстронг,_Скотт
Армстронг,_Джон
Пушка_Армстронга
Армстронг_(округ)
Армстронг_(фамилия)
Армстронг
Армстронг,_Дебби
Армстронг,_Дженни
Армстронг,_Джеймс_(борец)
Армстронг,_Генри_(боксёр)

Топ Hub: Армстронг
Армстронг,_Самира
Пушка_Армстронга
Армстронг,_Би_Джей
Армстронг,_Джо
Армстронг,_Джон
Армстронг,_Алан
Армстронг,_Дэвид
Армстронг,_Рой_Фрейзер
Армстронг,_Генри_(боксёр)
Армстронг_(округ,_Пенсильвания)
